In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Define the model save path
model_save_path = r"C:\Users\trejan\Desktop\GNN\Saved models\LogisticRegression.pkl"

# Ensure the output directory exists
output_dir = r"C:\Users\trejan\Desktop\GNN\XGB"
os.makedirs(output_dir, exist_ok=True)

#############################
# Pipeline 1: Logistic Regression for Obesity Risk Prediction
#############################

# Load genetic dataset (assumes comma-delimited)
genetic_file_path = r"C:\Users\trejan\Desktop\Sem 2\Machine Learning\model\new_genetic_profiles.csv"
genetic_df = pd.read_csv(genetic_file_path)
genetic_df.columns = genetic_df.columns.str.strip()
genetic_df.fillna("None", inplace=True)

# Convert Obesity_Risk_Score into categories (Low, Medium, High)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.5, 0.8, 1],
    labels=['Low', 'Medium', 'High']
)

# Initialize dictionary to store LabelEncoders
label_encoders = {}

# Encode categorical variables (Diet_Type, Physical_Activity)
for col in ["Diet_Type", "Physical_Activity"]:
    le = LabelEncoder()
    genetic_df[col] = le.fit_transform(genetic_df[col])
    label_encoders[col] = le

# Encode gene variant columns as strings (so that "None" is encoded too)
variant_columns = ["MC4R_Variant", "PPARG_Variant", "FTO_Variant", "LEPR_Variant"]
for col in variant_columns:
    genetic_df[col] = genetic_df[col].astype(str)
    le = LabelEncoder()
    genetic_df[col] = le.fit_transform(genetic_df[col])
    label_encoders[col] = le

# Define features and target for the genetic model
features = [
    "Age", "BMI", "Physical_Activity", "Diet_Type",
    "MC4R_Present", "MC4R_Variant",
    "PPARG_Present", "PPARG_Variant",
    "FTO_Present", "FTO_Variant",
    "LEPR_Present", "LEPR_Variant"
]
target = "Obesity_Risk_Category"

X_gen = genetic_df[features]
y_gen = genetic_df[target]

# Encode target labels (Low, Medium, High)
target_le = LabelEncoder()
y_encoded = target_le.fit_transform(y_gen)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_gen, y_encoded, test_size=0.2, random_state=42)

# Scale features (important for Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Logistic Regression model
logreg_model = LogisticRegression(
    multi_class='multinomial',  # For multi-class classification
    solver='lbfgs',            # Suitable for small datasets
    max_iter=1000,             # Increase iterations for convergence
    random_state=42
)
logreg_model.fit(X_train_scaled, y_train)

# Save the trained model
with open(model_save_path, 'wb') as model_file:
    pickle.dump(logreg_model, model_file)

print(f"Model saved successfully at {model_save_path}")

# Evaluate the model
y_pred_logreg = logreg_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_pred_logreg)
print("Logistic Regression Test Accuracy:", test_accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred_logreg, target_names=target_le.classes_))
print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred_logreg)
print(conf_matrix)

# Save metrics as a table
classification_report_dict = classification_report(y_test, y_pred_logreg, target_names=target_le.classes_, output_dict=True)
report_df = pd.DataFrame(classification_report_dict).transpose()

metrics_df = pd.DataFrame({
    'Metric': ['Test Accuracy'],
    'Value': [test_accuracy]
})

conf_matrix_df = pd.DataFrame(conf_matrix, columns=target_le.classes_, index=target_le.classes_)
conf_matrix_df['Metric'] = 'Confusion Matrix'

# Combine all metrics into a single DataFrame
metrics_table = pd.concat([metrics_df, report_df, conf_matrix_df], axis=0)

# Save the metrics table as a CSV file
metrics_table_path = os.path.join(output_dir, 'metrics_table_logreg.csv')
metrics_table.to_csv(metrics_table_path, index=False)
print(f"Metrics table saved to {metrics_table_path}")

# Plot and save the learning curve
train_sizes, train_scores, test_scores = learning_curve(logreg_model, X_train_scaled, y_train, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))

plt.figure()
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training score')
plt.plot(train_sizes, np.mean(test_scores, axis=1), label='Cross-validation score')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy Score')
plt.title('Learning Curves')
plt.legend(loc="best")
plt.savefig(os.path.join(output_dir, 'learning_curve_logreg.png'))
plt.close()

# Plot and save the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=target_le.classes_, yticklabels=target_le.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig(os.path.join(output_dir, 'confusion_matrix_logreg.png'))
plt.close()

#############################################
# Additional Evaluation Metrics for Logistic Regression
#############################################

# Binarize the test labels for ROC and Precision-Recall curves
n_classes = len(target_le.classes_)
y_test_bin = label_binarize(y_test, classes=list(range(n_classes)))
y_score = logreg_model.predict_proba(X_test_scaled)

# 1. Plot ROC Curves for each class
plt.figure()
colors = ['blue', 'red', 'green']
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=colors[i], lw=2,
             label=f"ROC curve of class {target_le.classes_[i]} (area = {roc_auc:0.2f})")
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
roc_curve_path = os.path.join(output_dir, 'roc_curve_logreg.png')
plt.savefig(roc_curve_path)
plt.close()
print(f"ROC curves saved to {roc_curve_path}")

# 2. Plot Precision-Recall Curves for each class
plt.figure()
for i in range(n_classes):
    precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
    avg_precision = average_precision_score(y_test_bin[:, i], y_score[:, i])
    plt.plot(recall, precision, lw=2,
             label=f'PR curve of class {target_le.classes_[i]} (AP = {avg_precision:0.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend(loc="upper right")
pr_curve_path = os.path.join(output_dir, 'precision_recall_curve_logreg.png')
plt.savefig(pr_curve_path)
plt.close()
print(f"Precision-Recall curves saved to {pr_curve_path}")

# 3. Plot Coefficient Bar Charts for each class
coefficients = logreg_model.coef_  # Shape: (n_classes, n_features)
for i, class_label in enumerate(target_le.classes_):
    plt.figure(figsize=(8, 6))
    plt.barh(range(len(features)), coefficients[i], align='center', color='skyblue')
    plt.yticks(range(len(features)), features)
    plt.xlabel("Coefficient Value")
    plt.title(f"Logistic Regression Coefficients for Class '{class_label}'")
    plt.tight_layout()
    coef_plot_path = os.path.join(output_dir, f'logreg_coefficients_{class_label}.png')
    plt.savefig(coef_plot_path)
    plt.close()
    print(f"Coefficient plot for class '{class_label}' saved to {coef_plot_path}")

#############################
# Pipeline 2: Meal Recommendation
#############################

# Load the meal dataset (assumed to be comma-delimited)
meal_file_path = r"C:\Users\trejan\Desktop\Sem 2\Machine Learning\model\train.csv"
meal_df = pd.read_csv(meal_file_path)

# Preprocess nutritional features; here we assume these columns exist
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using KMeans to create diverse groups (e.g., 10 clusters)
num_clusters = 10  # Use 10 clusters for more diversity
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Function to display meal clusters in a user-friendly format
def display_meal_cluster(meals, cluster_name, num_meals=5):
    print(f"\n=== Recommended Meals for {cluster_name} ===")
    print(f"Total Meals in Cluster: {len(meals)}")
    
    # Display a subset of meals
    print(f"\nTop {num_meals} Meals:")
    print(f"{'Meal Description':<50} {'Energy (kcal)':<15} {'Protein (g)':<15} {'Fat (g)':<15} {'Carbs (g)':<15}")
    print("-" * 110)
    for _, meal in meals.head(num_meals).iterrows():
        print(f"{meal['Descrip']:<50} {meal['Energy_kcal']:<15} {meal['Protein_g']:<15} {meal['Fat_g']:<15} {meal['Carb_g']:<15}")
    
    # Display summary of nutritional values
    avg_energy = meals['Energy_kcal'].mean()
    avg_protein = meals['Protein_g'].mean()
    avg_fat = meals['Fat_g'].mean()
    avg_carbs = meals['Carb_g'].mean()
    
    print("\n=== Nutritional Summary ===")
    print(f"Average Energy (kcal): {avg_energy:.2f}")
    print(f"Average Protein (g): {avg_protein:.2f}")
    print(f"Average Fat (g): {avg_fat:.2f}")
    print(f"Average Carbs (g): {avg_carbs:.2f}")

# Function to recommend meals dynamically based on user profile
def recommend_meals(user_profile, meal_df, model, label_encoder, num_meals=5):
    # Convert the user profile to a DataFrame
    user_profile_df = pd.DataFrame([user_profile])
    
    # Encode the categorical features in the user profile using stored encoders
    for col in ["Diet_Type", "Physical_Activity"]:
        if col in user_profile_df.columns:
            le = label_encoders[col]
            user_profile_df[col] = le.transform(user_profile_df[col])
    for col in variant_columns:
        if col in user_profile_df.columns:
            le = label_encoders[col]
            user_profile_df[col] = le.transform(user_profile_df[col].astype(str))
    
    # Ensure the user profile has all required features; add missing ones as 0 if needed
    missing_cols = set(features) - set(user_profile_df.columns)
    for col in missing_cols:
        user_profile_df[col] = 0
    user_profile_df = user_profile_df[features]
    
    # Scale the user profile features
    user_profile_scaled = scaler.transform(user_profile_df)
    
    # Predict the obesity risk category for the user profile
    predicted_category = model.predict(user_profile_scaled)[0]
    predicted_label = label_encoder.inverse_transform([predicted_category])[0]
    print(f"\nPredicted Obesity Risk Category: {predicted_label}")
    
    # Define cluster preferences based on the predicted risk
    if predicted_label == 'Low':
        # Low-risk users: Focus on high-protein, balanced meals
        preferred_clusters = [0, 1, 2]  # Example clusters
        sort_by = 'Protein_g'  # Sort by highest protein
    elif predicted_label == 'Medium':
        # Medium-risk users: Focus on moderate-calorie, balanced meals
        preferred_clusters = [3, 4, 5]  # Example clusters
        sort_by = 'Energy_kcal'  # Sort by moderate calories
    else:
        # High-risk users: Focus on low-calorie, nutrient-dense meals
        preferred_clusters = [6, 7, 8, 9]  # Example clusters
        sort_by = 'Energy_kcal'  # Sort by lowest calories
    
    # Recommend meals from the preferred clusters
    recommended_meals = meal_df[meal_df['Meal_Cluster'].isin(preferred_clusters)]
    
    # Sort meals based on the user's risk category
    if predicted_label == 'High':
        recommended_meals = recommended_meals.sort_values(by=sort_by, ascending=True)  # Low calories
    else:
        recommended_meals = recommended_meals.sort_values(by=sort_by, ascending=False)  # High protein or moderate calories
    
    # Display recommended meals in a user-friendly format
    display_meal_cluster(recommended_meals, f"{predicted_label}-Risk", num_meals)

# Example new genetic profile
new_profile = {
    "Age": 35,
    "BMI": 28.5,
    "Physical_Activity": "Low",      # Original string value (will be encoded)
    "Diet_Type": "High-Fat",           # Original string value (will be encoded)
    "MC4R_Present": 1,
    "MC4R_Variant": "rs17782313_TT",    # Original string value
    "PPARG_Present": 1,
    "PPARG_Variant": "rs1801282_CG",    # Original string value
    "FTO_Present": 1,
    "FTO_Variant": "rs9939609_AT",      # Original string value
    "LEPR_Present": 1,
    "LEPR_Variant": "rs1137101_AG"       # Original string value
}

# Recommend meals for the new profile
recommend_meals(new_profile, meal_df, logreg_model, target_le, num_meals=5)


C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model saved successfully at C:\Users\trejan\Desktop\GNN\Saved models\LogisticRegression.pkl
Logistic Regression Test Accuracy: 0.8075
Classification Report:
              precision    recall  f1-score   support

        High       0.89      0.94      0.92        88
         Low       0.75      1.00      0.85       182
      Medium       0.92      0.45      0.60       130

    accuracy                           0.81       400
   macro avg       0.85      0.80      0.79       400
weighted avg       0.83      0.81      0.79       400

Confusion Matrix:
[[ 83   0   5]
 [  0 182   0]
 [ 10  62  58]]
Metrics table saved to C:\Users\trejan\Desktop\GNN\XGB\metrics_table_logreg.csv
ROC curves saved to C:\Users\trejan\Desktop\GNN\XGB\roc_curve_logreg.png
Precision-Recall curves saved to C:\Users\trejan\Desktop\GNN\XGB\precision_recall_curve_logreg.png
Coefficient plot for class 'High' saved to C:\Users\trejan\Desktop\GNN\XGB\logreg_coefficients_High.png
Coefficient plot for class 'Low' saved to 

C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 556, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Age
- BMI
- Diet_Type
- FTO_Present
- FTO_Variant
- ...
Feature names seen at fit time, yet now missing:
- Carb_g
- Energy_kcal
- Fat_g
- Protein_g


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans
import random  # For introducing randomness in recommendations
import os  # To set environment variables

# Suppress joblib warning about physical cores
os.environ['LOKY_MAX_CPU_COUNT'] = '4'

#############################
# Pipeline 1: Logistic Regression for Obesity Risk Prediction
#############################

# Load genetic dataset (assumes comma-delimited)
genetic_file_path = r"C:\Users\trejan\Desktop\Sem 2\Machine Learning\model\new_genetic_profiles.csv"
genetic_df = pd.read_csv(genetic_file_path)
genetic_df.columns = genetic_df.columns.str.strip()
genetic_df.fillna("None", inplace=True)

# Convert Obesity_Risk_Score into categories (Low, Medium, High)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.5, 0.8, 1],
    labels=['Low', 'Medium', 'High']
)

# Initialize dictionary to store LabelEncoders
label_encoders = {}

# Encode categorical variables (Diet_Type, Physical_Activity)
for col in ["Diet_Type", "Physical_Activity"]:
    le = LabelEncoder()
    genetic_df[col] = le.fit_transform(genetic_df[col])
    label_encoders[col] = le

# Encode gene variant columns as strings (so that "None" is encoded too)
variant_columns = ["MC4R_Variant", "PPARG_Variant", "FTO_Variant", "LEPR_Variant"]
for col in variant_columns:
    genetic_df[col] = genetic_df[col].astype(str)
    le = LabelEncoder()
    genetic_df[col] = le.fit_transform(genetic_df[col])
    label_encoders[col] = le

# Define features and target for the genetic model
features = [
    "Age", "BMI", "Physical_Activity", "Diet_Type",
    "MC4R_Present", "MC4R_Variant",
    "PPARG_Present", "PPARG_Variant",
    "FTO_Present", "FTO_Variant",
    "LEPR_Present", "LEPR_Variant"
]
target = "Obesity_Risk_Category"

X_gen = genetic_df[features]
y_gen = genetic_df[target]

# Encode target labels (Low, Medium, High)
target_le = LabelEncoder()
y_encoded = target_le.fit_transform(y_gen)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_gen, y_encoded, test_size=0.2, random_state=42)

# Scale features (important for Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Logistic Regression model
logreg_model = LogisticRegression(
    solver='lbfgs',            # Suitable for small datasets
    max_iter=1000,             # Increase iterations for convergence
    random_state=42
)
logreg_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred_logreg = logreg_model.predict(X_test_scaled)
print("Logistic Regression Test Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("Classification Report:")
print(classification_report(y_test, y_pred_logreg, target_names=target_le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_logreg))

#############################
# Pipeline 2: Meal Recommendation (Same as Before)
#############################

# Load the meal dataset (assumed to be comma-delimited)
meal_file_path = r"C:\Users\trejan\Desktop\Sem 2\Machine Learning\model\train.csv"
meal_df = pd.read_csv(meal_file_path)

# Preprocess nutritional features; these columns should exist in your meal dataset
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler_meal = StandardScaler()
nutritional_features_scaled = scaler_meal.fit_transform(nutritional_features)

# Cluster meals using KMeans (e.g., 10 clusters)
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a meal recommendation function that uses the predicted obesity risk category
def recommend_meals(user_profile, meal_df, logreg_model, target_le, scaler, num_meals=5):
    """
    user_profile: dict with genetic feature values (original, unencoded)
    logreg_model: trained logistic regression model
    target_le: LabelEncoder for the target risk category
    scaler: StandardScaler fitted on genetic features
    """
    # Convert user_profile into a DataFrame
    user_profile_df = pd.DataFrame([user_profile])
    
    # Encode categorical features using stored encoders
    for col in ["Diet_Type", "Physical_Activity"]:
        if col in user_profile_df.columns:
            le = label_encoders[col]
            user_profile_df[col] = le.transform(user_profile_df[col])
    for col in variant_columns:
        if col in user_profile_df.columns:
            le = label_encoders[col]
            user_profile_df[col] = le.transform(user_profile_df[col].astype(str))
    
    # Ensure the user profile contains all required features; fill missing with 0
    missing_cols = set(features) - set(user_profile_df.columns)
    for col in missing_cols:
        user_profile_df[col] = 0
    user_profile_df = user_profile_df[features]
    
    # Scale the user profile using the same scaler as training
    user_profile_scaled = scaler.transform(user_profile_df)
    
    # Predict obesity risk using the logistic regression model
    predicted_category = logreg_model.predict(user_profile_scaled)[0]
    predicted_label = target_le.inverse_transform([predicted_category])[0]
    print(f"\nPredicted Obesity Risk Category: {predicted_label}")
    
    # Define cluster preferences based on predicted risk (example logic)
    if predicted_label == 'Low':
        preferred_clusters = [0, 1, 2, 3]  # Broaden the cluster selection
        sort_by = random.choice(['Protein_g', 'Energy_kcal'])  # Randomize sorting
        ascending = random.choice([True, False])  # Randomize order
    elif predicted_label == 'Medium':
        preferred_clusters = [4, 5, 6, 7]  # Broaden the cluster selection
        sort_by = random.choice(['Energy_kcal', 'Fat_g'])  # Randomize sorting
        ascending = random.choice([True, False])  # Randomize order
    else:
        preferred_clusters = [8, 9, 0, 1]  # Broaden the cluster selection
        sort_by = random.choice(['Energy_kcal', 'Carb_g'])  # Randomize sorting
        ascending = random.choice([True, False])  # Randomize order
    
    # Filter and sort meals from the preferred clusters
    recommended_meals = meal_df[meal_df['Meal_Cluster'].isin(preferred_clusters)]
    recommended_meals = recommended_meals.sample(frac=1).reset_index(drop=True)  # Shuffle the meals
    recommended_meals = recommended_meals.sort_values(by=sort_by, ascending=ascending)
    
    print("\nRecommended Meals:")
    print(recommended_meals[['Descrip', 'Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']].head(num_meals))

# Example new genetic profile for meal recommendation (using original, unencoded values)
new_profile = {
    "Age": 35,
    "BMI": 28.5,
    "Physical_Activity": "Low",      # Original string (will be encoded)
    "Diet_Type": "High-Fat",           # Original string (will be encoded)
    "MC4R_Present": 1,
    "MC4R_Variant": "rs17782313_TT",
    "PPARG_Present": 0,
    "PPARG_Variant": "rs1801282_CG",
    "FTO_Present": 1,
    "FTO_Variant": "rs9939609_AT",
    "LEPR_Present": 0,
    "LEPR_Variant": "rs1137101_AG"
}

# Get meal recommendations using the logistic regression model
recommend_meals(new_profile, meal_df, logreg_model, target_le, scaler, num_meals=5)

Logistic Regression Test Accuracy: 0.8075
Classification Report:
              precision    recall  f1-score   support

        High       0.89      0.94      0.92        88
         Low       0.75      1.00      0.85       182
      Medium       0.92      0.45      0.60       130

    accuracy                           0.81       400
   macro avg       0.85      0.80      0.79       400
weighted avg       0.83      0.81      0.79       400

Confusion Matrix:
[[ 83   0   5]
 [  0 182   0]
 [ 10  62  58]]

Predicted Obesity Risk Category: Medium

Recommended Meals:
                                                Descrip  Energy_kcal  \
2314     Egg, whole, dried, stabilized, glucose reduced        615.0   
364   Snacks, popcorn, oil-popped, microwave, regula...        583.0   
589                                   Egg, whole, dried        605.0   
1783          Puff pastry, frozen, ready-to-bake, baked        558.0   
2985          Candies, HERSHEY'S POT OF GOLD Almond Bar        577.0 